<a href="https://colab.research.google.com/github/ubinix-warun/colab-typhoon-llm-model-labs/blob/main/labs/quickstart_typhoon_ocr_python_demo_with_t4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! sudo apt update && sudo apt install pciutils lshw

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,795 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

This cell updates the package list and installs `pciutils` and `lshw`. These utilities are often used for inspecting hardware information.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve > ollama.log 2>&1 &

In [ ]:
! ollama run gemma3:12b “What is the capital of the Netherlands?”


The capital of the Netherlands is **Amsterdam**.



While The Hague is the seat of government, Amsterdam is officially the capital.



In [ ]:
!ollama ps

NAME          ID              SIZE     PROCESSOR    CONTEXT    UNTIL              
gemma3:12b    f4031aab637d    11 GB    100% GPU     4096       4 minutes from now    


In [ ]:
!ollama stop gemma3:12b

In [ ]:
!ollama rm gemma3:12b

deleted 'gemma3:12b'


In [ ]:
!ollama run scb10x/typhoon-ocr-3b:latest

In [ ]:
!pip install typhoon-ocr openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.9 MB/s eta 0:00:00


In [ ]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 1s (185 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126411 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import base64
from typhoon_ocr import prepare_ocr_messages
from openai import OpenAI

# Prepare messages for OCR processing
messages = prepare_ocr_messages(
    pdf_or_image_path="2301004319.pdf",  # Works with PDFs or images
    task_type="default",    # Choose between "default" or "structure"
    page_num=1              # Process page 1 of a PDF (default is 1, always 1 for images)
)

client = OpenAI(base_url="http://localhost:11434/v1", api_key="not-needed")

response = client.chat.completions.create(
    model="scb10x/typhoon-ocr-3b:latest",
    messages=messages,
    max_tokens=16000,
    extra_body={
        "repetition_penalty": 1.2,
        "temperature": 0.1,
        "top_p": 0.6,
    },
)

print(response.choices[0].message.content.natural_text)


{"natural_text": "(19) กรมทรัพย์สินทางปัญญา กระทรวงพาณิชย์\n\n(21) เลขที่คำขอ 2301004319\n(24) วันที่รับคำขอ 12 กรกฎาคม 2566\n(22) วันที่ยื่นคำขอ 14 มกราคม 2565\n\n(51) สัญลักษณ์จำแนกการประดิษฐ์ระหว่างประเทศ Int.CL.10\nC21D 9/00\n\n(72) ผู้ประดิษฐ์\nมิสิโนบุ ทากุยะ, ทาเกบายาชิ ฮิโรชิ, ทากาฮาชิ ทาเกฮิโร\n\n(74) ตัวแทน\nนางดารานีย์ วัจนะวุฒิวงศ์ และ/หรือ\nนางสาวกนกยา สังขพงศ์ และ/หรือ\nนางสาวกนกยาทิพย์ จินตุรินันท์ ซึ่งเป็นตัวแทนของ\nบริษัท ดิลิเก้นด์กิจยินส์ อินเตอร์เนชั่นแนล จำกัด เลขที่ 1011\nอาคารศุภาลัย แกรนด์ ทาวเวอร์ ชั้นที่ 20-26 ถ. พระราม 3\nแขวงของนนทรี เขตยานนาวา กรุงเทพมหานคร 10120\n\n(54) ชื่อที่แสดงถึงการประดิษฐ์\nชิ้นส่วนปั๊มร้อน\n\n(57) บทรูปการประดิษฐ์\n\nขั้นตอนการประดิษฐ์:\n\nขั้นตอนการประดิษฐ์ (hot stamped member) นี้รวมถึง: เหล็กกล้า และชิ้นชุบผิวซึ่งถูกสร้างขึ้นบนเหล็กกล้าดังกล่าว ในที่ซึ่งชิ้นชุบผิวมีองค์ประกอบทางเคมีที่ถูกกำหนดไว้ล่วงหน้า ชิ้นชุบผิวมีออกไซด์ที่มี Zn เป็นส่วนประกอบหลักที่รวมถึงหนึ่งหรือสองชนิดของ Zn ออกไซด์และ Zn-Mg ออกไซด์ ที่มีขนาดอยู่ที่ 1.0 ไมโ

In [ ]:
! sudo apt update && sudo apt install pciutils lshw

This cell downloads and installs the Ollama command-line tool using a shell script from the official Ollama website.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

This cell starts the Ollama server in the background using `nohup` and redirects the output to `ollama.log`. This allows the Ollama server to run without blocking the notebook execution.

In [ ]:
!nohup ollama serve > ollama.log 2>&1 &

This cell runs the `gemma3:12b` model using Ollama and asks it "What is the capital of the Netherlands?".

In [ ]:
! ollama run gemma3:12b “What is the capital of the Netherlands?”

This cell lists the running Ollama models.

In [ ]:
!ollama ps

This cell stops the `gemma3:12b` model running in Ollama.

In [ ]:
!ollama stop gemma3:12b

This cell removes the `gemma3:12b` model from Ollama.

In [ ]:
!ollama rm gemma3:12b

This cell downloads and unzips the ngrok executable.

In [ ]:
# Install ngrok
wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

This cell starts ngrok to create a tunnel for a local service running on port 8080.

In [ ]:
# Start ngrok for a local service on port 8080
./ngrok http 8080

This cell installs the `typhoon-ocr` and `openai` Python libraries using pip.

In [ ]:
!pip install typhoon-ocr openai

This cell installs the `poppler-utils` package using apt-get. This package is often required for working with PDF files.

In [ ]:
!apt-get install -y poppler-utils

This cell demonstrates how to use the `typhoon-ocr` library with the Ollama API to perform OCR on a PDF file. It prepares the messages for the API call and then sends the request to the local Ollama server.

In [ ]:
import base64
from typhoon_ocr import prepare_ocr_messages
from openai import OpenAI

# Prepare messages for OCR processing
messages = prepare_ocr_messages(
    pdf_or_image_path="2301004319.pdf",  # Works with PDFs or images
    task_type="default",    # Choose between "default" or "structure"
    page_num=1              # Process page 1 of a PDF (default is 1, always 1 for images)
)

client = OpenAI(base_url="http://localhost:11434/v1", api_key="not-needed")

response = client.chat.completions.create(
    model="scb10x/typhoon-ocr-3b:latest",
    messages=messages,
    max_tokens=16000,
    extra_body={
        "repetition_penalty": 1.2,
        "temperature": 0.1,
        "top_p": 0.6,
    },
)

print(response.choices[0].message.content.natural_text)

This cell attempts to extract and print the `natural_text` field from the `response` object. It tries to handle cases where the content might be a JSON string containing the `natural_text` field or just the plain text.

In [ ]:
import json

# Assuming 'response' is the object containing the chat completion result
# and 'natural_text' is a key within the 'content' of the message.
# The exact path might need adjustment based on the actual structure of 'response'.
try:
    # Assuming the natural_text is directly accessible as a string or needs to be loaded
    natural_text_content = response.choices[0].message.content
    try:
        # Attempt to parse as JSON and extract 'natural_text' if it's a JSON string
        natural_text_dict = json.loads(natural_text_content)
        if "natural_text" in natural_text_dict:
            print(natural_text_dict["natural_text"])
        else:
            # If it's JSON but doesn't have 'natural_text', print the original content
            print(natural_text_content)
    except json.JSONDecodeError:
        # If it's not a JSON string, print it directly
        print(natural_text_content)
except AttributeError:
    print("Could not find natural_text in the response.")